In [ ]:
import datetime
import six

print( "packages imported at " + str( datetime.datetime.now() ) )

In [ ]:
%run django_init.py

In [ ]:
# django model imports
from sourcenet.models import Article_Data
from context_datasets.models import DataSetCitationData
from context_datasets.models import DataSetMention

# other context_datasets classes
from context_datasets.coding.data_set_mentions.manual.manual_data_set_mentions_coder import ManualDataSetMentionsCoder

In [ ]:
# declare variables
article_data_qs = None
article_data = None
mention_qs = None
mention_count = None
coder_user = None
mention = None
mention_citation = None
citation_get_result = None
citation_data = None
citation_get_status = None
citation_get_message = None

# get all Article_Data.
article_data_qs = Article_Data.objects.all()

# loop
for article_data in article_data_qs:
    
    # see if any mentions.
    mention_qs = article_data.datasetmention_set.all()
    mention_count = mention_qs.count()
    if ( mention_count > 0 ):
        
        # retrieve user
        coder_user = article_data.coder
        
        # we have mentions.  For each:
        # - get citation
        # - pass Article_Data instance and DataSetCitation instance to
        #     ManualDataSetMentionsCoder.get_data_set_citation_data()
        #     to get or create DataSetCitationData.
        # - add that citation_data instance to mention.
        # - save mention.
        for mention in mention_qs:
            
            # get citation
            mention_citation = mention.data_set_citation
            
            # retrieve citation_data
            get_citation_data_result = ManualDataSetMentionsCoder.get_data_set_citation_data( article_data, mention_citation )

            # get citation data, status, status message.
            citation_data = get_citation_data_result.get( ManualDataSetMentionsCoder.PROP_CITATION_DATA, None )
            citation_get_status = get_citation_data_result.get( ManualDataSetMentionsCoder.PROP_LOOKUP_STATUS, ManualDataSetMentionsCoder.PROP_LOOKUP_STATUS_VALUE_ERROR )
            citation_get_message = get_citation_data_result.get( ManualDataSetMentionsCoder.PROP_STATUS_MESSAGE, None )

            # error?
            if ( citation_get_status not in ManualDataSetMentionsCoder.PROP_LOOKUP_ERROR_STATUS_LIST ):

                # no - add citation to mention and save.
                mention.data_set_citation_data = citation_data
                mention.save()

                # error.  No citation data.
                status_message = "Updated mention {} to refer to DataSetCitationData {}.".format( mention.id, citation_data.id )
                print( status_message )

            else:

                # error.  No citation data.
                status_message = "ERROR - status: {}; status message: {}.".format( citation_get_status, citation_get_message )
                print( status_message )

            # END check to see if citation present. --#
            
        #-- END loop over mentions --#
        
    #-- END check to see if mentions --#
        
#-- END loop over Article_Data --#